In [20]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI 
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
import os 
# Load environment variables from .env file
load_dotenv()

# Access API keys
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

system_prompt = '''You are a helpful finance expert named Wakasiaka in year 2024. First of all you create a plan to get answer to the research query. 
Then you use tools to get answers to the questions. Finally you use the answers to each question in the plan to give your final verdict.'''

llm = ChatOpenAI(model = "gpt-4o-mini",openai_api_key=openai_api_key )
tools = [TavilySearchResults(max_results = 3, tavily_api_key= tavily_api_key)]
agent_executor = create_react_agent(llm, tools, state_modifier = system_prompt)


In [14]:
import operator
from pydantic import BaseModel, Field
from typing import Annotated,List, Tuple
from typing_extensions import TypedDict

class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    plan_steps: Annotated[List[Tuple], operator.add]
    response: str

class Plan(BaseModel):
    """ plan to follow in the future """

    steps: List[str] = Field(
        description= "diffeerent steps to follow, should be in sorted order"
    )


In [22]:
from langchain_core.prompts import ChatPromptTemplate
planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ You are a finance research agent working in Oct 2024. For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer.
 Do not add any superfluous steps. The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps. 
At the end use the info collected to give the final answer to the main question containing the facts."""
        ),
        ("placeholder", "{messages}"),
    ]
)

In [24]:
planner = planner_prompt | ChatOpenAI(
    model = "gpt-4o-mini", temperature= 0,openai_api_key = openai_api_key
).with_structured_output(Plan)

planner.invoke(
    {
        "messages": [
            ("user", "Should we Invest in tesla given the curret situation of EV?")
        ]
    }
)



AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: KKK. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List

# Define Plan Schema
class Plan(BaseModel):
    """Plan to follow in the future"""
    steps: List[str] = Field(
        description="Different steps to follow, should be in sorted order"
    )

# Define Prompt Template
planner_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are a finance research agent working in Oct 2024. 
        For the given objective, come up with a simple step-by-step plan.
        This plan should involve individual tasks, that if executed correctly, will yield the correct answer.
        Do not add any superfluous steps. The result of the final step should be the final answer.
        Make sure that each step has all the information needed—do not skip steps.
        At the end, use the collected info to give the final answer to the main question containing the facts."""
    ),
    ("user", "{messages}"),
])

# Use API Key in ChatOpenAI
planner = planner_prompt | ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=openai_api_key  # Pass API key explicitly
).with_structured_output(Plan)

# Invoke Planner
result = planner.invoke({
    "messages": [{"role": "user", "content": "Should we invest in Tesla given the current situation of EV?"}]
})

# Display Result
print(result)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: KKK. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}